In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 읽어와 시스템 환경 변수에 추가

api_key = os.getenv("GEMINI_API_KEY")
vanna_api_key = os.getenv("MY_VANNA_API_KEY")


from vanna.vannadb import VannaDB_VectorStore
from vanna.google import GoogleGeminiChat

In [2]:
class MyVanna(VannaDB_VectorStore, GoogleGeminiChat):
    def __init__(self, config=None):
        MY_VANNA_MODEL = 'hosung_test' # Your model name from https://vanna.ai/account/profile
        VannaDB_VectorStore.__init__(self, vanna_model=MY_VANNA_MODEL, vanna_api_key=vanna_api_key, config=config)
        GoogleGeminiChat.__init__(self, config={'api_key': api_key, 'model': 'gemini-2.0-flash-exp'})

vn = MyVanna()

/Users/hosung/anaconda3/envs/vanna/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vn.connect_to_mysql(host='127.0.0.1', dbname='hosungdb', user='root', password='hosung-secret', port=3306)


In [5]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

In [9]:
# The following are methods for adding training data. Make sure you modify the examples to match your database.

# DDL statements are powerful because they specify table names, colume names, types, and potentially relationships
vn.train(ddl="""
    CREATE TABLE IF NOT EXISTS my-table (
        id INT PRIMARY KEY,
        name VARCHAR(100),
        age INT
    )
""")


# Sometimes you may want to add documentation about your business terminology or definitions.
vn.train(documentation="Our business defines OTIF score as the percentage of orders that are delivered on time and in full")

# You can also add SQL queries to your training data. This is useful if you have some queries already laying around. You can just copy and paste those from your editor to begin generating new SQL.
vn.train(sql="SELECT * FROM my-table WHERE name = 'John Doe'")

TypeError: 'NoneType' object is not iterable

In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

In [ ]:
training_data

In [ ]:
training_data

In [ ]:
# You can remove training data if there's obsolete/incorrect information. 
vn.remove_training_data(id='455835-ddl')

In [ ]:
vn.ask(question=...)

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()